In [19]:
from flask import Flask, render_template, url_for, request, redirect
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
import numpy as np
import pandas as pd

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///attendance.db'
db = SQLAlchemy(app)

class Professor(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50), nullable=False)
    username = db.Column(db.String(50), nullable=False)
    password = db.Column(db.String(50), nullable=False)
    subjectid = db.Column(db.String(20), nullable=False)
    
    def __repr__(self):
        return f"Professor('{self.name}', '{self.username}', '{self.password}')"

with app.app_context():
    db.create_all()
    
    newuser1 = Professor(id=4, name="Professor 4", username="prof4", password="prof4", subjectid="2002_Digital Portfolio")
    #newuser2 = Professor(id=2, name="Professor 2", username="prof2", password="prof2", subjectid="2001_Knowledge and expert System")
    db.session.add(newuser1)
    #db.session.add(newuser2)
    db.session.commit()
    
df = pd.DataFrame({'RollNo': [1001, 1002, 1003, 1004],
                   'Name': ['ABCDEF', 'XYZ', 'MNOP', 'STUVW'],
                   'Attendance': ['Present', 'Absent', 'Present', 'Absent']})

@app.route('/')    
@app.route('/login', methods=['POST', 'GET'])
def login():
    msg = ''
    if request.method == 'POST' and 'username' in request.form and 'password' in request.form:
        username = request.form['username']
        password = request.form['password']
        account = Professor.query.filter_by(username=username, password=password).first()
        if account:
            return redirect('/show/'+account.subjectid)
        else:
            msg = 'Incorrect username / password!'
    return render_template('login.html', msg=msg)

@app.route('/show/<subjectid>', methods=['GET'])
def show(subjectid):
    print(subjectid)
    date = datetime.today().strftime('%m-%d-%Y')
    if 'rollno' in locals():
        print(rollno)
        return redirect('/show')
    return render_template('show.html', rows=df.iterrows(), date=date)

@app.route('/change/<int:rollno>')
def change(rollno):
    if df.loc[df['RollNo'] == rollno, 'Attendance'].iloc[0] == 'Present':
        df.loc[df['RollNo'] == rollno, 'Attendance'] = 'Absent'
    else:
        df.loc[df['RollNo'] == rollno, 'Attendance'] = 'Present'
    return redirect('/show')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
import pyodbc
import pandas as pd
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};'
                      'SERVER=tcp:facercgntn.database.windows.net;'
                      'DATABASE=Attendance_Database;UID=face_recognition;PWD=fnrcgntn123@#;')
student_id = '1004' #input value
class_id = ''
cursor = cnxn.cursor()
query = "Select student.student_id, student_name, professor_name, subject_name, course_name, class_id from student " \
        "left join timetable on student.student_id = timetable.student_id " \
        "left join course_subject on timetable.subject_id = course_subject.subject_id " \
        "left join professor on timetable.professor_id = professor.professor_id " \
        "where student.student_id = " + student_id +\
        "and day = DATENAME(weekday, GETDATE()) and ABS(DATEDIFF(MINUTE, GETDATE(), start_time)) <= 30 "

cursor.execute(query)
# for row in cursor:
#     print(row)

# Create a list of column names
columns = [column[0] for column in cursor.description]

# Create a list of rows
rows = cursor.fetchall()

# Create a dataframe from the rows and columns
df = pd.DataFrame.from_records(rows, columns=columns)
name_column = df['class_id']
print(name_column[0])

In [ ]:
!pip install pyodbc

In [ ]:
newuser1 = Professor(id=1, name="Professor 1", username="prof1", password="prof1", subjectid="2000_Applied ML")
newuser2 = Professor(id=1, name="Professor 2", username="prof2", password="prof2", subjectid="2001_Knowledge and expert System")
db.session.add(newuser1)
db.session.add(newuser2)
db.session.commit()

In [2]:
from flask import Flask, render_template, url_for, request, redirect
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
import numpy as np
import pandas as pd
import pyodbc

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///attendance.db'
db = SQLAlchemy(app)

class Professor(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50), nullable=False)
    username = db.Column(db.String(50), nullable=False)
    password = db.Column(db.String(50), nullable=False)
    subjectid = db.Column(db.String(20), nullable=False)
    
    def __repr__(self):
        return f"Professor('{self.name}', '{self.username}', '{self.password}')"

with app.app_context():
    db.create_all()
    
cnxn = pyodbc.connect('DRIVER={SQL Server};'
                      'SERVER=tcp:facercgntn.database.windows.net;'
                      'DATABASE=Attendance_Database;UID=face_recognition;PWD=fnrcgntn123@#;')
cursor = cnxn.cursor()
    

@app.route('/')    
@app.route('/login', methods=['POST', 'GET'])
def login():
    msg = ''
    if request.method == 'POST' and 'username' in request.form and 'password' in request.form:
        username = request.form['username']
        password = request.form['password']
        account = Professor.query.filter_by(username=username, password=password).first()
        if account:
            return redirect('/show/'+account.subjectid)
        else:
            msg = 'Incorrect username / password!'
    return render_template('login.html', msg=msg)

@app.route('/show/<subjectid>', methods=['GET'])
def show(subjectid):
    date = datetime.today().strftime('%m-%d-%Y')
    query = "Select student_id, student_name, subject_name, status, day, attendance_capture_time " \
            "from [attendance] " \
            "where day = DATENAME(weekday, GETDATE()) and " \
            "subject_name = '" +subjectid+ "'"
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]
    rows = cursor.fetchall()
    df1 = pd.DataFrame.from_records(rows, columns=columns)
    global df 
    df = df1
    print(df)
    return render_template('show.html', rows=df.iterrows(), date=date)

@app.route('/change/<int:rollno>')
def change(rollno):
    print(rollno)
    print(df)
    subject_name = df.loc[df['student_id'] == rollno, 'subject_name'].iloc[0]
    print(subject_name)
    if df.loc[df['student_id'] == rollno, 'status'].iloc[0] == 'Present':
        df.loc[df['student_id'] == rollno, 'status'] = 'Absent'
        status = "Absent"
        att_time = "NULL"
        df.loc[df['student_id'] == rollno, 'attendance_capture_time'] = att_time
    else:
        df.loc[df['student_id'] == rollno, 'status'] = 'Present'
        status = "Present"
        att_time = "'"+datetime.now().strftime('%Y-%m-%d %H:%M:%S')+"'"
        df.loc[df['student_id'] == rollno, 'attendance_capture_time'] = att_time
    query = "UPDATE [attendance]" \
            "SET status = '" +status+ "', attendance_capture_time = "+att_time+ \
            " WHERE student_id = " +str(rollno)+ " and subject_name = '" +subject_name+ "'"
    cursor.execute(query)
    cnxn.commit()
    return redirect('/show/'+subject_name)
 
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Apr/2023 16:18:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2023 16:18:39] "GET /static/login.css HTTP/1.1" 200 -


In [ ]:
pip install flask_sqlalchemy

In [ ]:
 pip install Flask-SQLAlchemy